<a href="https://colab.research.google.com/github/garcia-sebastianf/Contrato-cadena-de-ahorros/blob/main/MapeoReglas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Script para mapeo de reglas**
                        

In [21]:
#@title Librerias

!pip install pyxlsb openpyxl

import openpyxl
from google.colab import runtime
from openpyxl.utils import column_index_from_string
from IPython.display import display
from openpyxl import load_workbook
from openpyxl.styles import NamedStyle
import numpy as np
import pandas as pd
import os

In [22]:
# Script que realiza el proceso de mapeo de reglas y genera un nuevo archivo de salida con los resultados.

# INSTRUCCIONES

# Es recomendable comprobar que el resultado sea correcto comparando TOTAL de reglas técnicas y de dominio del resultado con lo que
# aparece en el backlog. Esto se debe a que podrían presentarse errores o inconsistencias en el backlog, y en ese caso, sería necesario
# recurrir juicio humano para corregirlos.


## En esta sección simplemente cambian los valores según la necesidad.

############# Namings tabla mapeo de reglas ###############################################
nombre_tabla = 't_kctd_interaction_activity'
status = 'Lista para implementar reglas'

# nombre_proyecto = 'Compliance'
# nombre_proyecto = 'Alpha'
nombre_proyecto = 'Client solutions - NGA'

############# Namings tabla de backlog ####################################################

# El backlog es el único archivo que se debe importar a Colab
# Para importar archivo, en la parte derecha, selecciona Files -> Upload to session storage

nombre_archivo_backlog = 'Backlog Unico DQ C360.xlsx' # Client solutions - NGA
# nombre_archivo_backlog = 'Backlog Único DQ Compliance.xlsx'
# nombre_archivo_backlog = 'Backlog Unico DQ ALPHA -domains-.xlsx'
# nombre_archivo_backlog = 'Backlog Unico DQ ALPHA.xlsx'

# nombre_hoja = 'Backlog Unico'# Compliance
nombre_hoja = 'Backlog Unico Final' # Client solutions

#  Normalmente estos valores están por defecto
columna_TABLE = 'AB'
columna_TypeofRule = 'P'
columna_NAMING = 'V'
columna_catalog = 'AL'
columna_tecnical_domain = 'R'

# Para ejecutar el script: Runtime -> Run all
# El archivo de salida se mostrara en la sección de Files
NombreArchivoSalida = 'MapeoReglas_'+ nombre_proyecto + '_' + nombre_tabla

In [23]:
#@title Programa

#Inicialización de las variables del programa

directory = '/content/'
nombre_col_catalog, nombre_col_TypeofRule, nombre_col_NAMING, nombre_col_tecnical_domain = '','','',''
datos_backlog = pd.DataFrame()
reglas_mapeadas = pd.DataFrame()
Flag = True
i = 0
j = 0
num_reglas_tecnicas = 0
num_reglas_dominio = 0
num_reglas_tecnicas_campo = 0
num_reglas_dominio_campo = 0


# Función para generar las constantes de columnas de Excel
def generar_constantes_excel(n):
    columnas_excel = {}
    for i in range(n):
        column = ''
        temp = i
        while temp >= 0:
            column = chr(temp % 26 + 65) + column
            temp = temp // 26 - 1
        columnas_excel[column] = i
    return columnas_excel


# Generar columnas hasta la 150
COLUMNAS_EXCEL = generar_constantes_excel(150)


# Obtención del valor numérico de la columna.
def col(columna):
    return COLUMNAS_EXCEL.get(columna.upper(), "Columna no válida")


# Retorna los valores de la columna
def inx(table, columna):
  return table[table.columns[col(columna)]]


# Maneja la lógica de impresión de las reglas mapeadas
def Cond_impresion(string1 , string2, condicion):
  if condicion == 0:
    return '- ' + string1 + string2 + '\n'
  else:
    return string1 + '- ' + string2


# Registra la acomulación de las reglas técnicas mapeadas
def Regla_tec(valor_mapeado):
  regla_tecnica =  reglas_mapeadas.loc[j,'Reglas Técnicas']
  condicion = i
  return Cond_impresion(regla_tecnica , datos_backlog.loc[i,nombre_col_TypeofRule], condicion)


# Registra la acomulación de reglas de dominio mapeadas
def Regla_dom(valor_mapeado):
  regla_dominio =  reglas_mapeadas.loc[j,'Reglas de Dominio']
  condicion = i
  return Cond_impresion(regla_dominio , datos_backlog.loc[i,nombre_col_TypeofRule] , condicion)


# Registra la acomulación de reglas de dominio mapeadas más el catálogo
def Regla_dom_catalog(valor_mapeado):
  regla_dominio =  reglas_mapeadas.loc[j,'Reglas de Dominio']
  catalogo = " " + '(' + datos_backlog.loc[i,nombre_col_catalog] + ')'
  condicion = i
  return Cond_impresion(regla_dominio , datos_backlog.loc[i,nombre_col_TypeofRule] + catalogo , condicion)


# Obtener backlog y guardarlo en un DataFrame
def Obtencion_backlog():
  global datos_backlog
  for file in os.listdir(directory):
    if nombre_archivo_backlog in file:
      datos_backlog = pd.read_excel(os.path.join(directory, file),
                                      sheet_name = nombre_hoja,
                                      #usecols='A:BQ',
                                      dtype = 'string',
                                      decimal = ',',
                                      skiprows=2)


#Filtrar campos a partir del nombre de la tabla.
def Filtrar_campos():
  global datos_backlog
  datos_backlog = datos_backlog[inx(datos_backlog, columna_TABLE) == nombre_tabla ]


# Obtener valor numérico de las reglas.
def Obtener_reglas():
  global datos_backlog
  datos_backlog[datos_backlog.columns[col(columna_TypeofRule)]] = inx(datos_backlog, columna_TypeofRule).str[:3]


# Obtener y guardar el nombre de las columnas
def Obtener_nombre_col():
  global nombre_col_catalog, nombre_col_TypeofRule, nombre_col_NAMING, nombre_col_tecnical_domain
  nombre_col_catalog = datos_backlog.columns[col(columna_catalog)]
  nombre_col_TypeofRule = datos_backlog.columns[col(columna_TypeofRule)]
  nombre_col_NAMING = datos_backlog.columns[col(columna_NAMING)]
  nombre_col_tecnical_domain = datos_backlog.columns[col(columna_tecnical_domain)]


# Filtrar las columnas de interés para realizar el mapeo
def Filtrar_columnas():
  global datos_backlog
  datos_backlog = datos_backlog.iloc[:, [col(columna_TypeofRule), col(columna_NAMING),
                                       col(columna_tecnical_domain), col(columna_catalog)]]

# Establecer las reglas de objeto
def Def_reglas_objeto():
  global datos_backlog
  datos_backlog = datos_backlog.reset_index(drop=True) #[]
  datos_backlog.loc[datos_backlog.isin({nombre_col_TypeofRule: ['4.2','6.9']})[nombre_col_TypeofRule], nombre_col_NAMING ] = "Objeto"


# Crear DataFrame vacio para la tabla de salida (con las reglas mapeadas)
def Iniciar_tabla_mapeada():
  global reglas_mapeadas
  # Lista de nombres de las columnas
  columnas = ['Tablas', 'Status', 'Proyecto','Total de campos relevantes',
            'En Ejercicio Governance Holding','Total reglas',
            'Campos (En verde los campos relevantes)','Reglas de Dominio',
            'Reglas Técnicas','Revisión en el Portal']
  # Crear un DataFrame vacío con esas columnas
  reglas_mapeadas = pd.DataFrame(columns=columnas)
  reglas_mapeadas.loc[0] = [nombre_tabla, status, nombre_proyecto,'','','','','','','']


# Iniciar las funciones para dar inicio a la ejecución del programa.
Obtencion_backlog()
Filtrar_campos()
Obtener_reglas()
Obtener_nombre_col()
Filtrar_columnas()
Def_reglas_objeto()
Iniciar_tabla_mapeada()

# Esto es algo parecido a un "polling"
while(Flag):

  num_filas = datos_backlog.shape[0]
  num_reglas_tecnicas_campo = 0
  num_reglas_dominio_campo = 0


  for i in range(num_filas):
    campo_origen = datos_backlog.loc[0,nombre_col_NAMING]

    if campo_origen  == datos_backlog.loc[i,nombre_col_NAMING]:
      if j > 0 and i == 0:
        reglas_mapeadas.loc[j] = ['','','','','','',campo_origen ,'','','']

      elif j == 0 and i == 0:
        reglas_mapeadas.loc[j,'Campos (En verde los campos relevantes)'] = campo_origen

      if datos_backlog.loc[i,nombre_col_tecnical_domain] == 'Technical':
        reglas_mapeadas.loc[j,'Reglas Técnicas'] = Regla_tec(reglas_mapeadas.loc[j,'Reglas Técnicas'])
        num_reglas_tecnicas = num_reglas_tecnicas + 1
        num_reglas_tecnicas_campo = num_reglas_tecnicas_campo + 1

      else:
        if pd.isna(datos_backlog.at[i, nombre_col_catalog]):
          reglas_mapeadas.loc[j,'Reglas de Dominio'] = Regla_dom(reglas_mapeadas.loc[j,'Reglas de Dominio'])
          num_reglas_dominio = num_reglas_dominio + 1
          num_reglas_dominio_campo = num_reglas_dominio_campo + 1
        else:
          reglas_mapeadas.loc[j,'Reglas de Dominio'] = Regla_dom_catalog(reglas_mapeadas.loc[j,'Reglas de Dominio'])
          num_reglas_dominio = num_reglas_dominio + 1
          num_reglas_dominio_campo = num_reglas_dominio_campo + 1

  if num_reglas_tecnicas_campo == 1:
    reglas_mapeadas.loc[j,'Reglas Técnicas'] = reglas_mapeadas.loc[j,'Reglas Técnicas'].rstrip('\n')

  if num_reglas_dominio_campo == 1:
    reglas_mapeadas.loc[j,'Reglas de Dominio'] = reglas_mapeadas.loc[j,'Reglas de Dominio'].rstrip('\n')

  cond_eliminacion = datos_backlog.isin({nombre_col_NAMING:[campo_origen]})[nombre_col_NAMING]
  datos_backlog.drop(cond_eliminacion[cond_eliminacion].index , inplace=True)
  datos_backlog = datos_backlog.reset_index(drop=True)

  j = j + 1
  numero_campos = j + 1
  if datos_backlog.empty:
    Flag = False

reglas_mapeadas.loc[0,'Total reglas'] = "Dominio:" + str(num_reglas_dominio) + "\n" + "Técnicas:" + str(num_reglas_tecnicas)

reglas_mapeadas.to_excel('/content/'+ NombreArchivoSalida+ ".xlsx", index=False)

# Guardar el DataFrame en un archivo Excel
file_path = '/content/'+ NombreArchivoSalida+ ".xlsx"



# Escribir el DataFrame en el archivo Excel
with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
    reglas_mapeadas.to_excel(writer, index=False, sheet_name='Reglas')

    workbook = writer.book
    worksheet = workbook['Reglas']

    for col in worksheet.columns:
        max_length = 0
        column = col[0].column_letter
        for cell in col:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass

        adjusted_width = max_length + 1
        worksheet.column_dimensions[column].width = adjusted_width

    for row in worksheet.iter_rows():
        for cell in row:

            cell.number_format = "@"
            if isinstance(cell.value, str) and '\n' in cell.value:
              cell.alignment = openpyxl.styles.Alignment(wrap_text=True)

    num_inicio_rango = 2
    num_fin_rango = numero_campos
    columnas = ['A', 'B', 'C', 'D', 'E', 'F']

    for columna in columnas:
      rango = f"{columna}{num_inicio_rango}:{columna}{num_fin_rango}"
      inicio_rango = f"{columna}{num_inicio_rango}"

      worksheet.merge_cells(rango)
      alignment = openpyxl.styles.Alignment( vertical='center')
      worksheet[inicio_rango].alignment = alignment

    for row in worksheet.iter_rows(min_col=6, max_col=6, min_row=2, max_row=numero_campos):
      for cell in row:
        alignment = openpyxl.styles.Alignment(vertical='center', wrap_text=True)
        cell.alignment = alignment

# Borrar todas las variables en el entorno de Google Colab y reinicia el kernell
!colab.reboot()

/bin/bash: -c: line 2: syntax error: unexpected end of file
